#### Loading Data from SKAB Github

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
from scipy.stats import gaussian_kde
import plotly.figure_factory as ff
# from utility_functions import *
# from models import *
from scipy.spatial import distance
import umap
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # This is needed for 3D plotting
import plotly.graph_objs as go
from plotly.offline import iplot
import random
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
import os
import math

c:\Users\VSIE43\AppData\Local\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# random.seed(42)
# np.random.seed(42)
# torch.manual_seed(42)

In [ ]:
s=42
random.seed(s)
np.random.seed(s)
torch.manual_seed(s)

In [ ]:
data_pth = r'C:\Users\VSIE43\OneDrive - Scania CV\SKAB-master\data/'
normal_file = 'anomaly-free/anomaly-free.csv'
test_1 = 'other'
test_2 = 'valve1'
test_3 = 'valve2'

df = pd.read_csv(data_pth+normal_file, sep=';')

In [ ]:
df.columns 

In [ ]:
def fillin_time_gaps(df):
    df.datetime = pd.to_datetime(df.datetime.values)
    time_diff = np.diff(df.datetime.values)

    # there will be need for data imputation. 
    # some samples are with differnce of 2 seconds, rather than 1 second
    new_time = pd.date_range(df.datetime.min(), df.datetime.max(),freq='1s')
    missing_time = pd.DataFrame({'datetime' : new_time})
    df_new = missing_time.merge(df, on='datetime', how='left')

    # maybe fill in with interpolation
    df_new = df_new.interpolate(method='ffill')
    return df_new

In [ ]:
normal_data = []
df = pd.read_csv(data_pth+normal_file, sep=';')
normal_data.append(fillin_time_gaps(df).drop(columns=['datetime']))
for folder in [test_2, test_3]:
    files = os.listdir(data_pth+folder)
    print(files)
    for file in files:
        tmp = pd.read_csv(data_pth+folder+'/'+file, sep=';')
        # print(tmp.columns)
        # t = tmp[tmp.anomaly==1]
        t=tmp[tmp.anomaly==0]
        t = t.drop(columns=['datetime','anomaly','changepoint'])
        normal_data.append(t)

In [ ]:
df = fillin_time_gaps(df)

In [ ]:
def min_max_normalize(df, m_m_params=None):
    if m_m_params:
        (min_p, max_p) = m_m_params
    else:
        (min_p, max_p) = df.min(), df.max()

    new_df = (df-min_p) / (max_p-min_p)

    return new_df,  (min_p, max_p)


# data, m_m_params = min_max_normalize(df.drop(columns=['datetime']))
# ind = math.floor(0.8*len(data))
# train_data = data[:ind]
# val_data = data[ind:]
# # del data

In [ ]:
def create_time_window(X, look_back=64):
    dataX = []
    for i in range(len(X)-look_back-1):
        a = X[i:(i+look_back), :]
        dataX.append(a)
    return np.array(dataX)

# train_x = create_time_window(train_data.values)
# val_x = create_time_window(val_data.values)

In [ ]:
# creating data from more data

train_ext_x  = []
val_ext_x = []
tmp = []
[tmp.append(create_time_window(t.values)) for t in normal_data]
for t in tmp:
    ind = math.floor(0.8*len(t))
    val_ext_x.append(t[ind:])
    train_ext_x.append(t[:ind])

# [print(tmp.shape) for tmp in train_ext_x]
train_ext_x = np.concatenate(train_ext_x, axis=0)
new_m_m_parms =  train_ext_x.min(axis=(0,1)), train_ext_x.max(axis=(0,1))
train_ext_x,_ = min_max_normalize(train_ext_x, new_m_m_parms)

val_ext_x = np.concatenate(val_ext_x, axis=0)
val_ext_x,_ = min_max_normalize(val_ext_x, new_m_m_parms)


# l_ind = len(train_ext_x)-math.floor(len(train_ext_x)*.8)
# rand_index = np.random.randint(0,len(train_ext_x),l_ind)

# val_ext_x = train_ext_x[rand_index]
# train_ext_x = np.delete(train_ext_x,rand_index)
# val_ext_x = train_ext_x[ind:]
# train_ext_x = train_ext_x[:ind]

In [ ]:
val_ext_x.shape

In [ ]:
train_ext_x.shape

In [ ]:
def array_to_dataloader(data, batch_size, shuffle):

    data_tensor = torch.tensor(data, dtype=torch.float32)
    print(data_tensor.transpose(1,2).shape)
    dataset_ae = TensorDataset(data_tensor.transpose(1,2), data_tensor.transpose(1,2))
    dataloader_ae = DataLoader(dataset_ae, batch_size=batch_size, shuffle=shuffle)

    return dataloader_ae

In [ ]:
train_dataloader = array_to_dataloader(train_ext_x, batch_size=32, shuffle=False)
val_dataloader = array_to_dataloader(val_ext_x, batch_size=32, shuffle=False)

In [ ]:
def plot_data(data):
    num_columns = len(data.columns)
    nrows = num_columns  # One row for each column

    # Decrease the height allocated to each subplot by reducing the second element of the figsize tuple.
    # Adjust the 5 in figsize=(10, 5) based on your preference and the actual number of subplots.
    height_per_subplot = 1  # Adjust this value to change the height per subplot
    fig = plt.figure(figsize=(10, height_per_subplot * nrows))
    
    # Create subplots with adjusted spacing using 'subplots_adjust' if necessary.
    # The 'hspace' parameter controls the height of the padding between subplots.
    gs = fig.add_gridspec(nrows, 1, hspace=0.35)  # Adjust hspace as needed
    
    for i, column in enumerate(data.columns):
        ax = fig.add_subplot(gs[i, 0])

        # Plot the line on the ith subplot
        ax.plot(data[column], label=f'Signal_{i+1}')
        
        # Set y-axis to show only the min and max values
        col_min = data[column].min()
        col_max = data[column].max()
        ax.set_ylim(col_min, col_max)
        ax.set_yticks([col_min, col_max])  # Show only ticks for min and max
        
        # Format the y-tick labels
        ax.yaxis.set_major_formatter(plt.FormatStrFormatter('%.2f'))
        
        # Set title aligned to the left
        ax.set_title(f'Signal_{i+1}', loc='left', fontsize=10, verticalalignment='center')
        
        # Show legend with column name
        # ax.legend(loc='upper right')

        # Make x-axis ticks visible only for the last subplot
        if i < num_columns - 1:
            ax.set_xticklabels([])
        else:
            ax.set_xlabel('Time')
            ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))

    # Adjust the layout to prevent overlap and save the figure
    plt.tight_layout()
    # plt.savefig('compact_subplots.png')
    plt.show()

In [ ]:
test_df_1 = pd.read_csv(data_pth+test_1+'/1.csv',sep=';')
test_df_1_norm, _  = min_max_normalize(test_df_1.drop(columns=['datetime','anomaly','changepoint']),new_m_m_parms)
test_x = create_time_window(test_df_1_norm.values)

In [ ]:
plot_data(test_df_1_norm)

In [ ]:
test_x.shape

In [ ]:
test_dataloader = array_to_dataloader(test_x, batch_size=32, shuffle=False)

### Anomaly Detection - Conv1D Variational Autoencoder

In [ ]:
class Conv1DVAE(nn.Module):
    def __init__(self, num_features):
        super(Conv1DVAE, self).__init__()

        self.total_features = num_features
        # Adjusted for num_features and seq_length
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=self.total_features, out_channels=32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding=2),
            nn.ReLU()
        )
        
        # Adjusted h_dim based on the output size of the last Conv1d layer, which may need fine-tuning
        self.fc1 = nn.Linear(64, 64)  # Adjust for a more gradual reduction
        self.fc2 = nn.Linear(64, 32)  # Further reduce dimensionality
        self.fc3 = nn.Linear(32, 16)  # mu layer, increased from 8 to 16
        self.fc4 = nn.Linear(32, 16)  # logvar layer, increased from 8 to 16
        self.fc5 = nn.Linear(16, 32)  # For decoding, start expansion
        self.fc6 = nn.Linear(32, 64)  # Continue expansion
        self.fc7 = nn.Linear(64, 64)  # Prepare for decoder convolution layers
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=32, out_channels=self.total_features, kernel_size=5, padding=2),
            nn.ReLU()
        )

    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling
        return sample


    def encode(self, x):
    # def encode(self, x, k=10): # x ->(1, 64, 10) x_hat->(k, 64, 10)
    #  return z , mu, log_var # z ->(k, 64, l)
        

        x = self.encoder(x)
        
        h = self.fc1(x)
        h = self.fc2(h)

        mu = self.fc3(h)
        
        logvar = self.fc4(h)

        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def decode(self, z):
        
        z = self.fc5(z)  # Prepare z for the decoder
        z = self.fc6(z)
        z = self.fc7(z)

        x_recon = self.decoder(z)
        
        x_recon = x_recon.transpose(1,2)
       
        return x_recon

    def forward(self, x):

        z, mu, logvar = self.encode(x)
        x_recon = self.decode(z)
        # Return the reconstructed outputs, along with the latent variables for loss calculation
        return x_recon.transpose(1,2), mu, logvar

In [ ]:
class Conv1DVAE(nn.Module):
    def __init__(self, num_features):
        super(Conv1DVAE, self).__init__()
        self.total_features = num_features

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=self.total_features, out_channels=32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding=2),
            nn.ReLU()
        )

        # Fully connected layers for mu and logvar
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)  # mu layer
        self.fc4 = nn.Linear(32, 16)  # logvar layer

        # Decoder FC layers
        self.fc5 = nn.Linear(16, 32)
        self.fc6 = nn.Linear(32, 64)
        self.fc7 = nn.Linear(64, 64)

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=32, out_channels=self.total_features, kernel_size=5, padding=2),
            nn.ReLU()
        )

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + (eps * std)

    def encode(self, x, k=10):
        batch_size, _, seq_length = x.size()
        mu_list = []
        logvar_list = []
        z_list = []

        for i in range(k):
            x_encoded = self.encoder(x)
            h = self.fc1(x_encoded)
            h = self.fc2(h)
            mu = self.fc3(h)
            logvar = self.fc4(h)
            z = self.reparameterize(mu, logvar)
            mu_list.append(mu)
            logvar_list.append(logvar)
            z_list.append(z)

        # Stack on new dimension to return tensors of shape [batch_size, num_features, seq_length, k]
        mu_stack = torch.stack(mu_list, dim=3)
        logvar_stack = torch.stack(logvar_list, dim=3)
        z_stack = torch.stack(z_list, dim=3)
        return z_stack, mu_stack, logvar_stack

    def decode(self, z):
        # Assuming z shape [batch_size, num_features, seq_length, k]
        batch_size, num_features, seq_length, k = z.size()
        x_recon_list = []

        for i in range(k):
            sample = z[:, :, :, i]  # Get the ith sample across all batches
            z_processed = self.fc5(sample)
            z_processed = self.fc6(z_processed)
            z_processed = self.fc7(z_processed)
            x_recon = self.decoder(z_processed)
            x_recon_list.append(x_recon)

        # Stack the reconstructions along the last dimension to match the desired output shape
        x_recon_stack = torch.stack(x_recon_list, dim=3)
        return x_recon_stack

    def forward(self, x, k=10):
        z, mu, logvar = self.encode(x, k)
        x_recon = self.decode(z)
        return x_recon, mu, logvar

#### Model Instantiation

In [ ]:
tot_num_features = train_ext_x.shape[2]
tot_num_features

In [ ]:
from torch.optim.lr_scheduler import StepLR

vae_model = Conv1DVAE(num_features=tot_num_features)
# vae_model = ConvVAE()
vae_optimizer = optim.Adam(vae_model.parameters(), lr=0.0001, amsgrad=True)
scheduler_vae = StepLR(vae_optimizer, step_size=10, gamma=0.5)

##### Custom Loss Function

In [ ]:
# def loss_fn(pred, target, mu, logvar):
#     # BCE = F.binary_cross_entropy(bin_output, bin_target)
#     mseLoss = nn.MSELoss()

#     loss = mseLoss(pred, target)
#     # see Appendix B from VAE paper:
#     # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
#     # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
#     KLD =  -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

#     return loss + (0.1*KLD), loss, KLD

In [ ]:
def loss_fn(pred, target, mu, logvar, k):
    mseLoss = nn.MSELoss()
    
    # Calculate MSE loss for each of the k samples and average them
    loss = torch.mean(torch.stack([mseLoss(pred[..., i], target) for i in range(k)], dim=0))
    
    # Compute KLD for each of the k samples and average them
    KLD = -0.5 * torch.mean(torch.mean(1 + logvar - mu.pow(2) - logvar.exp(), dim=[0, 1, 2, 3]))

    return loss + (0.1 * KLD), loss, KLD


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming `vae_model` and `vae_optimizer` have been defined
vae_model = vae_model.to(device)
vae_optimizer = optim.Adam(vae_model.parameters(), lr=0.0001, amsgrad=True)

In [ ]:
num_epochs = 100
train_losses, val_losses = [], []
train_cont_losses, train_bin_losses = [], []
val_cont_losses, val_bin_losses = [], []

# Initialize the variable to track the lowest validation loss
best_val_loss = float('inf')
best_model_path_vae = 'C:/Users/VSIE43/OneDrive - Scania CV/Models/best_model_vae_skab_2May_new_seed'+str(s)+'.pth'

for epoch in range(num_epochs):
    vae_model.train()  # Set the model to training mode
    train_loss = 0.0
    for data, target in train_dataloader:
        data, target = data.to(device), target.to(device)
        output, mu, logvar = vae_model(data)
        loss_elbo, loss, kld = loss_fn(output, target, mu, logvar, k=10)

        vae_optimizer.zero_grad()
        loss_elbo.backward()
        vae_optimizer.step()

        train_loss += loss_elbo.item()

    avg_train_loss = train_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)

    vae_model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for data, target in val_dataloader:
            data, target = data.to(device), target.to(device)
            output, mu, logvar = vae_model(data)
            loss_elbo_val, loss, kld = loss_fn(output, target, mu, logvar, k=10)
            val_loss += loss_elbo_val.item()

    avg_val_loss = val_loss / len(val_dataloader)
    val_losses.append(avg_val_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(vae_model.state_dict(), best_model_path_vae)
        print(f"Epoch {epoch+1}: New best model saved with loss {avg_val_loss:.4f}")

# Plotting training and validation losses
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss')
plt.title('Training vs. Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
best_model_path_vae = 'C:/Users/VSIE43/OneDrive - Scania CV/Models/best_model_vae_skab_2May_new_seed'+str(s)+'.pth'
best_model_vae = vae_model  # Assuming 'model' is an instance of your model class
best_model_vae.load_state_dict(torch.load(best_model_path_vae))
print("Loaded the best model.")
print(best_model_vae)

In [ ]:
def reconstruct_original_sequences(dataloader):
    original_sequences = []

    # Iterate over both dataloaders simultaneously
    for data, _ in dataloader:

        original_sequences.append(data.cpu().numpy())

    original_sequences = np.concatenate(original_sequences, axis=0)
    
    return original_sequences

In [ ]:
original_train_sequences = reconstruct_original_sequences(train_dataloader)
original_val_sequences = reconstruct_original_sequences(val_dataloader)
original_test_sequences = reconstruct_original_sequences(test_dataloader)
# original_test_sequences_rearWheel = reconstruct_original_sequences(anom_dataloader_rearWheel)
print("Original Train Shape:", original_train_sequences.shape)
print("Original Validation  Shape:", original_val_sequences.shape)
print("Original Test Shape:", original_test_sequences.shape)

In [ ]:
def plot_vae_model_predictions(model_vae, dataloader, original_sequences, window_index, feature_index):
    model_vae.eval()  # Ensure the model is in evaluation mode
    all_predicted_signals = []

    with torch.no_grad():  # No gradient needed for evaluation
        # Loop through all batches in the DataLoader
        for data, target in dataloader:
            # Predict using the model for each batch
            pred,_,_ = model_vae(data)
            # Concatenate predictions for each batch
            all_predicted_signals.append(pred)
        
        # Concatenate all predictions across batches
        all_predicted_signals = torch.cat(all_predicted_signals, dim=0)
    all_predicted_signals = np.array(all_predicted_signals)
    # Ensure the concatenated predictions match the original sequences' shape
    print(f"Predicted Signals Shape: {all_predicted_signals.shape}")
    print(f"Original Sequences Shape: {original_sequences.shape}")

    # Plotting (adjust as necessary for your specific needs)
    # Example: Plotting the first feature of the first sequence
    # plt.figure(figsize=(15, 5))
    # plt.plot(original_sequences[window_index, feature_index, :], label='Original Signal', marker='o')  # Adjust indexing as per your data shape
    # plt.plot(all_predicted_signals[window_index, feature_index, :], label='Reconstructed Signal', marker='x')  # Adjust indexing as per your data shape
    # # plt.ylim(0,1)
    # plt.title('Original vs. Reconstructed Signals')
    # plt.xlabel('Window Length')
    # plt.ylabel('Feature Value')
    # plt.savefig('vae_recon_plot.png', bbox_inches='tight', dpi=1000)
    # plt.legend()
    # plt.show()
    return all_predicted_signals

In [ ]:
def plot_vae_model_predictions(model_vae, dataloader, device):
    model_vae.eval()  # Ensure the model is in evaluation mode
    all_predicted_signals = []

    with torch.no_grad():  # No gradient needed for evaluation
        for data, target in dataloader:
            # Move data to the appropriate device
            data = data.to(device)

            # Predict using the model for each batch
            pred, _, _ = model_vae(data)
            
            # Append predictions (with the k dimension intact)
            all_predicted_signals.append(pred)

        # Concatenate all predictions across batches
        all_predicted_signals = torch.cat(all_predicted_signals, dim=0)
        print(f"Predicted Signals Shape: {all_predicted_signals.shape}")
    return all_predicted_signals.cpu().numpy()

In [ ]:
# train_recon_vae_signals = plot_vae_model_predictions(best_model_vae, train_dataloader, original_train_sequences, 1500, 5)
# valid_recon_vae_signals = plot_vae_model_predictions(best_model_vae, val_dataloader, original_val_sequences, 1000, 5)
# test_recon_vae_signals = plot_vae_model_predictions(best_model_vae, test_dataloader, original_test_sequences, 100, 5)

In [ ]:
train_recon_vae_signals = plot_vae_model_predictions(best_model_vae, train_dataloader, device)
# valid_recon_vae_signals = plot_vae_model_predictions(best_model_vae, val_dataloader, device)
# test_recon_vae_signals = plot_vae_model_predictions(best_model_vae, test_dataloader, device)

In [ ]:
# train_loss_mae_mse_vae = (train_recon_vae_signals - original_train_sequences)**2 + np.abs(train_recon_vae_signals - original_train_sequences)
# valid_loss_mae_mse_vae = (valid_recon_vae_signals - original_val_sequences)**2 + np.abs(valid_recon_vae_signals - original_val_sequences)
# test_loss_mae_mse_vae = (test_recon_vae_signals - original_test_sequences)**2 + np.abs(test_recon_vae_signals - original_test_sequences)
# # test_loss_mae_mse_ae_rearWheel = (test_recon_ae_signals_rearWheel[:,:20,:] - original_test_sequences_rearWheel[:,:20,:])**2 + np.abs(test_recon_ae_signals_rearWheel[:,:20,:] - original_test_sequences_rearWheel[:,:20,:])

# print('Train_cont Window Losses:', train_loss_mae_mse_vae.shape)
# print('Valid_cont Window Losses:', valid_loss_mae_mse_vae.shape)
# print('Test_cont Window Losses:', test_loss_mae_mse_vae.shape)
# # print('Test_cont Window Losses:', test_loss_mae_mse_ae_rearWheel.shape)

In [ ]:
train_recon_vae_signals.shape[3]

In [ ]:
def calculate_residuals_for_each_k_sample(recon_signals, original_signals):
    # Ensure recon_signals shape: [batch_size, num_features, seq_length, k]
    # Original signals should be expanded to match the shape for broadcasting
    original_signals_expanded = np.expand_dims(original_signals, axis=-1)
    
    # Initialize an array to hold combined losses for all k samples
    combined_losses = np.zeros_like(recon_signals)

    # Compute MSE and MAE for each sample, store results in combined_losses
    for i in range(recon_signals.shape[3]):
        mse = (recon_signals[:,:,:,i] - original_signals) ** 2
        mae = np.abs(recon_signals[:,:,:,i] - original_signals)
        combined_losses[:,:,:,i] = mse + mae

    return combined_losses

# Example Usage
train_loss_mae_mse_vae = calculate_residuals_for_each_k_sample(train_recon_vae_signals, original_train_sequences)
valid_loss_mae_mse_vae = calculate_residuals_for_each_k_sample(valid_recon_vae_signals, original_val_sequences)
test_loss_mae_mse_vae = calculate_residuals_for_each_k_sample(test_recon_vae_signals, original_test_sequences)

print('Train Loss:', train_loss_mae_mse_vae.shape)
print('Valid Loss:', valid_loss_mae_mse_vae.shape)
print('Test Loss:', test_loss_mae_mse_vae.shape)

In [ ]:
# train_sample_loss_vae = np.mean(train_loss_mae_mse_vae, axis=2).mean(axis = 1)
# valid_sample_loss_vae = np.mean(valid_loss_mae_mse_vae, axis=2).mean(axis = 1)
# test_sample_loss_vae = np.mean(test_loss_mae_mse_vae, axis=2).mean(axis = 1)

# print('Train_cont Sample Losses:', train_sample_loss_vae.shape)
# print('Valid_cont Sample Losses:', valid_sample_loss_vae.shape)
# print('Test_cont Sample Losses:', test_sample_loss_vae.shape)

In [ ]:
train_sample_loss_vae = np.mean(train_loss_mae_mse_vae, axis=2).mean(axis = 1)
valid_sample_loss_vae = np.mean(valid_loss_mae_mse_vae, axis=2).mean(axis = 1)
test_sample_loss_vae = np.mean(test_loss_mae_mse_vae, axis=2).mean(axis = 1)

print('Train_cont Sample Losses:', train_sample_loss_vae.shape)
print('Valid_cont Sample Losses:', valid_sample_loss_vae.shape)
print('Test_cont Sample Losses:', test_sample_loss_vae.shape)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_sample_losses(losses, title):
    """Plots boxplots for each k sample's losses."""
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=losses)
    plt.title(title)
    plt.xlabel('Sample Index (k)')
    plt.ylabel('Loss')
    plt.show()

# Plotting the sample losses for training, validation, and testing sets
plot_sample_losses(train_sample_loss_vae, 'Training Sample Losses Distribution')
plot_sample_losses(valid_sample_loss_vae, 'Validation Sample Losses Distribution')
plot_sample_losses(test_sample_loss_vae, 'Test Sample Losses Distribution')


In [ ]:
train_feature_loss_vae = np.mean(train_loss_mae_mse_vae, axis=2)
valid_feature_loss_vae = np.mean(valid_loss_mae_mse_vae, axis=2)
test_feature_loss_vae = np.mean(test_loss_mae_mse_vae, axis=2)

print('Train_cont Feature Losses:', train_feature_loss_vae.shape)
print('Valid_cont Feature Losses:', valid_feature_loss_vae.shape)
print('Test_cont Feature Losses:', test_feature_loss_vae.shape)

In [ ]:
Q1 = np.percentile(valid_sample_loss_vae, 25)
Q3 = np.percentile(valid_sample_loss_vae, 75)

# Calculate the Interquartile Range (IQR)
IQR = Q3 - Q1

# Determine the threshold as Q3 + multiplier * IQR
threshold_vae = Q3 + 1.5 * IQR
threshold_vae

In [ ]:
threshold_vae = np.percentile(valid_sample_loss_vae, 97.5)
threshold_vae

In [ ]:
# threshold_vae = np.percentile(valid_sample_loss_vae, 95)
# sns.histplot(train_losses, kde=True, color='red', alpha=0.2, edgecolor='black')
fig = sns.histplot(valid_sample_loss_vae, kde=True, color='blue', alpha=0.3, edgecolor='black')
fig.axvline(x = threshold_vae, linestyle = 'dashed', color = 'red')
# sns.histplot(test_losses, kde=True, color='red', alpha=0.2, edgecolor='black')

In [ ]:
plt.plot(test_sample_loss_vae)
# plt.plot(test_df_1.anomaly)
plt.figure()
plt.plot(test_sample_loss_vae>0.6)
plt.plot(test_df_1.anomaly.values)

In [ ]:
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics

In [ ]:
 # metrics

def  f1_score(true, pred):
    return metrics.f1_score(true, pred)

def recall_score(true, pred):
    return metrics.recall_score(true, pred)

def auc_score(true, pred):
    return metrics.roc_auc_score(true, pred)

def fpr_score(true, pred):
    tn, fp, fn, tp = confusion_matrix(true, pred).ravel()
    # print(c.shape)
    # FP = c.sum(axis=0) - np.diag(c)  
    # FN = c.sum(axis=1) - np.diag(c)
    # TP = np.diag(c)
    # TN = c.sum() - (FP + FN + TP)

    return fp/(fp+tn)

In [ ]:
# preds = []
# labels = []
# file_number = []
# # m = model_ext


# files = os.listdir(data_pth+test_1)

# print(files)
# for file in files:
#     tmp = pd.read_csv(data_pth+test_1+'/'+file, sep=';')
#     t = tmp.anomaly.values
#     test_df_1_norm, _  = min_max_normalize(tmp.drop(columns=['datetime','anomaly','changepoint']).values,new_m_m_parms)
#     test_x = create_time_window(test_df_1_norm)
#     test_dataloader = array_to_dataloader(test_x, batch_size=32, shuffle=False)
#     original_test_sequences = reconstruct_original_sequences(test_dataloader)
#     test_recon_ae_signals = plot_vae_model_predictions(best_model_vae, 
#                                                test_dataloader, 
#                                                original_test_sequences, 
#                                                window_index=600, 
#                                                feature_index=5)
#     test_loss_mae_mse_ae = (test_recon_ae_signals - original_test_sequences)**2 + np.abs(test_recon_ae_signals - original_test_sequences)
#     test_sample_loss_ae = np.mean(test_loss_mae_mse_ae, axis=2).mean(axis = 1)
#     labels.append(t[-len(test_recon_ae_signals):])
#     i = int(file.split('.')[0])
#     file_number.append([i]*len(test_recon_ae_signals))
#     preds.append(test_sample_loss_ae>threshold_vae)
#     # i +=1
    
# preds = np.concatenate(preds, axis=0)
# labels = np.concatenate(labels, axis=0)
# file_number = np.concatenate(file_number, axis=0)

In [ ]:
# preds = []
# labels = []
# file_number = []
# # m = model_ext


# files = os.listdir(data_pth+test_1)

# print(files)
# for file in files:
#     tmp = pd.read_csv(data_pth+test_1+'/'+file, sep=';')
#     t = tmp.anomaly.values
#     test_df_1_norm, _  = min_max_normalize(tmp.drop(columns=['datetime','anomaly','changepoint']).values,new_m_m_parms)
#     test_x = create_time_window(test_df_1_norm)
#     test_dataloader = array_to_dataloader(test_x, batch_size=32, shuffle=False)
#     original_test_sequences = reconstruct_original_sequences(test_dataloader)
#     test_recon_ae_signals = plot_vae_model_predictions(best_model_vae, 
#                                                test_dataloader, 
#                                                device)
#     test_loss_mae_mse_ae = calculate_residuals_for_each_k_sample(test_recon_ae_signals, original_test_sequences)
#     test_sample_loss_ae = np.mean(test_loss_mae_mse_ae, axis=2).mean(axis = 1)
#     labels.append(t[-len(test_recon_ae_signals):])
#     i = int(file.split('.')[0])
#     file_number.append([i]*len(test_recon_ae_signals))
#     preds.append(test_sample_loss_ae>threshold_vae)
#     # i +=1
    
# preds = np.concatenate(preds, axis=0)
# labels = np.concatenate(labels, axis=0)
# file_number = np.concatenate(file_number, axis=0)

In [ ]:
preds = []
labels = []
file_number = []
# m = model_ext


files = os.listdir(data_pth+test_1)

print(files)
for file in files:
    tmp = pd.read_csv(data_pth+test_1+'/'+file, sep=';')
    t = tmp.anomaly.values
    test_df_1_norm, _  = min_max_normalize(tmp.drop(columns=['datetime','anomaly','changepoint']).values,new_m_m_parms)
    test_x = create_time_window(test_df_1_norm)
    test_dataloader = array_to_dataloader(test_x, batch_size=32, shuffle=False)
    original_test_sequences = reconstruct_original_sequences(test_dataloader)
    test_recon_ae_signals = plot_vae_model_predictions(best_model_vae, 
                                               test_dataloader, 
                                               device)
    test_loss_mae_mse_ae = calculate_residuals_for_each_k_sample(test_recon_ae_signals, original_test_sequences)
    test_sample_loss_ae = np.mean(test_loss_mae_mse_ae, axis=2).mean(axis = 1)
    labels.append(t[-len(test_recon_ae_signals):])
    i = int(file.split('.')[0])
    file_number.append([i]*len(test_recon_ae_signals))
    preds.append(test_sample_loss_ae>threshold_vae)
    # i +=1
    
preds = np.concatenate(preds, axis=0)
labels = np.concatenate(labels, axis=0)
file_number = np.concatenate(file_number, axis=0)

In [ ]:
# # Assuming fpr_score function is defined as before
# def fpr_score(true, pred):
#     tn, fp, fn, tp = confusion_matrix(true, pred).ravel()
#     return fp / (fp + tn) if (fp + tn) != 0 else 0

# # Metric functions list
# metrics_functions = [f1_score, fpr_score, recall_score, auc_score]
# metric_names = ['f1_score', 'fpr_score', 'recall_score', 'auc_score']

# # Anomaly group ranges
# anomaly_ranges = {
#     'Fluid Leak Anomaly': range(1, 5),
#     'Rotor Imbalance Anomaly': range(5, 10),
#     'low/Sudden Increase in Water Anomaly': range(10, 12),
#     'Cavitation Anomaly': range(12, 14),
#     'High Temperature Anomaly': range(14, 15)
# }

# # Iterate over each metric function
# for i, fun in enumerate(metrics_functions):
#     print()
#     print(metric_names[i])
    
#     # Iterate over each anomaly group
#     for anomaly_name, indices in anomaly_ranges.items():
#         scores = [fun(labels[file_number == f], preds[file_number == f]) for f in indices]
#         average_score = np.mean(scores)
#         print(f"{anomaly_name} Average: {average_score:.4f}")

In [ ]:
anomaly_ranges = {
    'Fluid Leak Anomaly': range(1, 5),
    'Rotor Imbalance Anomaly': range(5, 10),
    'Low/Sudden Increase in Water Anomaly': range(10, 12),
    'Cavitation Anomaly': range(12, 14),
    'High Temperature Anomaly': range(14, 15)
}

# Collect scores
all_scores = {name: [] for name in anomaly_ranges.keys()}

# Iterate over each anomaly group
for anomaly_name, indices in anomaly_ranges.items():
    # Collect scores for each k sample
    scores_per_k = []
    for k in range(preds.shape[1]):
        scores = [f1_score(labels[file_number == f], preds[file_number == f, k]) for f in indices]
        scores_per_k.append(scores)
    all_scores[anomaly_name].append(scores_per_k)

# Visualize scores
for anomaly_name, scores in all_scores.items():
    plt.figure(figsize=(10, 4))
    for i, k_scores in enumerate(scores):
        plt.plot(k_scores, label=f'Sample {i}')
    plt.title(f'F1 Scores for {anomaly_name}')
    plt.xlabel('File Index')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.show()

In [ ]:
tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()

In [ ]:
fp / (tp+fp)

In [ ]:
ind = np.in1d(file_number, list(range(5,15)))
# print(np.unique(ind))
recall_score(labels[ind],preds[ind])

In [ ]:
f1_score(labels,preds)

In [ ]:
for i, fun in enumerate([f1_score, fpr_score, recall_score, auc_score]):
    print()
    print(['f1_score', 'fpr_score', 'recall_score', 'auc_score'][i])
    for f in range(1,15):
        print(fun(labels[file_number==f],preds[file_number==f]))

In [ ]:
print('FPR - Fluid Leak: ', (0.09411+0.05882+0.07669+0.06185+0.08407)/5)
print('FPR - Rotor Imbalance: ', (0.09411+0.05882+0.07669+0.06185+0.08407)/5)
print('FPR - Rotor Imbalance: ', (0.09411+0.05882+0.07669+0.06185+0.08407)/5)
print('FPR - Rotor Imbalance: ', (0.09411+0.05882+0.07669+0.06185+0.08407)/5)
print('FPR - Rotor Imbalance: ', (0.09411+0.05882+0.07669+0.06185+0.08407)/5)

In [ ]:
def create_counterfactuals_adam_pytorch(dataloader, model, threshold=-np.inf, learning_rate=0.01, num_iterations=1000, exclude_signals=[]):
    model.eval()
    
    all_counterfactuals = []
    loss_lst = []
    all_losses = []  # List to store losses for each batch

    for batch_idx, (data, targe) in enumerate(dataloader):
        # Ensure data is on the correct device and enable gradient
        data = data.clone().detach().requires_grad_(True)
        

        # Create optimizers for both continuous and binary data
        optimizer_data = torch.optim.Adam([data], lr=learning_rate)
        batch_losses = []
        for i in range(num_iterations):
            optimizer_data.zero_grad()


            output, mu, logvar = model(data)
            
            # Compute the loss
            loss_cont = torch.mean(F.mse_loss(output, data, reduction='mean') + -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp()))
            if loss_cont.item() < threshold:
                break

            loss_cont.backward()
            batch_losses.append(loss_cont.item())

            # Exclude specific signals from the gradient update
            if exclude_signals:
                with torch.no_grad():
                    for exclude_signal in exclude_signals:
                        data.grad[:, exclude_signal, :] = 0
                        

            optimizer_data.step()
            

        all_counterfactuals.append(data.detach().cpu().numpy())
        all_losses.append(batch_losses)

        # Optional: Plot loss per batch or just keep track of it
        plt.plot(loss_lst)
        plt.xlabel('Iteration')
        plt.ylabel('Loss')
        plt.title(f'Loss over Iterations for Batch {batch_idx}')
        plt.show()
        # loss_lst = []  # Reset loss list for the next batch

    # Combine all batches into a single numpy array
    all_counterfactuals = np.concatenate(all_counterfactuals, axis=0)
    

    return all_counterfactuals, all_losses

In [ ]:
all_feature_indices = list(range(tot_num_features))
validity_datasets = []
sparsity_datasets = []
proximity_datasets = []
for i in range(5,6):
    print()
    print('File number: ', i)
    test_df_1 = pd.read_csv(data_pth+test_1+'/'+str(i)+'.csv',sep=';')
    test_df_1_norm, _  = min_max_normalize(test_df_1.drop(columns=['datetime','anomaly','changepoint']).values,new_m_m_parms)
    test_x = create_time_window(test_df_1_norm)
    test_dataloader = array_to_dataloader(test_x, batch_size=32, shuffle=False)
    original_test_sequences = reconstruct_original_sequences(test_dataloader)
    test_recon_ae_signals = plot_vae_model_predictions(best_model_vae, 
                                               test_dataloader, 
                                               original_test_sequences, 
                                               window_index=600, 
                                               feature_index=5)
    test_loss_mae_mse_ae = (test_recon_ae_signals - original_test_sequences)**2 + np.abs(test_recon_ae_signals - original_test_sequences)
    test_sample_loss_ae = np.mean(test_loss_mae_mse_ae, axis=2).mean(axis = 1)
    test_feature_loss_ae = np.mean(test_loss_mae_mse_ae, axis=2)
    # sns.heatmap(test_feature_loss_ae)
    # sns.pairplot(test_df_1.drop(columns=['datetime','changepoint']), hue='anomaly')
    # scaler = MinMaxScaler()
    lof = LocalOutlierFactor(n_neighbors=2, contamination=0.4, algorithm='auto', metric='euclidean')
    feature_losses_transposed = test_feature_loss_ae.T
    # feature_losses_transposed_scaled = scaler.fit_transform(feature_losses_transposed)
    anomalies = lof.fit_predict(feature_losses_transposed)

    # Anomalies are marked with -1, so we can find them like this:
    anomalous_feature_indices = np.where(anomalies == -1)[0]
    
    print("Anomalous Feature Indices:", anomalous_feature_indices)

    exclude_signals = [index for index in all_feature_indices if index not in anomalous_feature_indices]

    print("Exclude Signals Mask:", exclude_signals)
    # plt.plot(test_sample_loss_ae)
    # plt.figure()
    # plt.plot(test_sample_loss_ae>ae_threshold)
    # plt.plot(test_df_1.anomaly.values[-len(test_sample_loss_ae):])
    # plt.show()
    cf_anom, losses = create_counterfactuals_adam_pytorch(dataloader=test_dataloader, 
                                            model=best_model_vae, 
                                            num_iterations=350, 
                                            learning_rate=0.01,exclude_signals=exclude_signals)

    cf_data_tensor = torch.tensor(cf_anom, dtype=torch.float32)
    cf_dataset = TensorDataset(cf_data_tensor, cf_data_tensor)
    cf_dataloader = DataLoader(cf_dataset, batch_size=32, shuffle=False)
    original_cf_sequences = reconstruct_original_sequences(cf_dataloader)
    recon_cf_signals = plot_vae_model_predictions(best_model_vae, 
                                            cf_dataloader,
                                            original_cf_sequences,
                                            195,
                                            2)

    cf_loss_mae_mse_ae = (recon_cf_signals - original_cf_sequences)**2 + np.abs(recon_cf_signals - original_cf_sequences)
    cf_sample_loss_ae = np.mean(cf_loss_mae_mse_ae, axis=2).mean(axis = 1)

    print('CF_cont Sample Losses:', cf_sample_loss_ae.shape)
    valid_counterfactuals = np.mean(cf_sample_loss_ae < threshold_vae)
    print(f'File Number {i} and its corresponding validity: {valid_counterfactuals}')
    validity_datasets.append(valid_counterfactuals)
    distances_per_timestep = np.sqrt(np.sum((original_test_sequences - cf_anom) ** 2, axis=2))

    average_distances_per_sequence = np.mean(distances_per_timestep, axis=1)

    overall_average_distance_per_window = np.mean(average_distances_per_sequence)
    print(f'File Number {i} and its corresponding distance metric: {overall_average_distance_per_window}')
    proximity_datasets.append(overall_average_distance_per_window)

    change_threshold = 0.001

    differences_exceed_threshold = np.abs(original_test_sequences - cf_anom) > change_threshold

    sparsity_per_sequence = np.sum(differences_exceed_threshold, axis=(1, 2)) / (original_test_sequences.shape[1] * original_test_sequences.shape[2])

    overall_average_sparsity = np.mean(sparsity_per_sequence)
    print(f'File Number {i} and its corresponding sparsity metric: {overall_average_sparsity}')
    sparsity_datasets.append(overall_average_sparsity)

In [ ]:
def get_anomalous_classified_window(model, threshold, norm_parameter=new_m_m_parms, files=range(1,15), is_fp_tp=False):
    data = []
    file_number = []
    fps = []
    tps = []
    for i in files:
        print(i)
        test_df_1 = pd.read_csv(data_pth+test_1+'/'+str(i)+'.csv',sep=';')
        label= test_df_1['anomaly'][65:] 
        test_df_1_norm, _  = min_max_normalize(test_df_1.drop(columns=['datetime','anomaly','changepoint']).values,norm_parameter)
        test_x = create_time_window(test_df_1_norm)
        test_dataloader = array_to_dataloader(test_x, batch_size=32, shuffle=False)
        original_test_sequences = reconstruct_original_sequences(test_dataloader)
        test_recon_ae_signals = plot_vae_model_predictions(model, 
                                                test_dataloader, 
                                                original_test_sequences, 
                                                window_index=600, 
                                                feature_index=5)
        test_loss_mae_mse_ae = (test_recon_ae_signals - original_test_sequences)**2 + np.abs(test_recon_ae_signals - original_test_sequences)
        test_sample_loss_ae = np.mean(test_loss_mae_mse_ae, axis=2).mean(axis = 1)
        t = test_sample_loss_ae>threshold
        data.append(test_x[t])
        file_number.append([i]*len(data[-1]))
        fps.append(np.logical_and(label==0,t==1)[t==1])
        tps.append(np.logical_and(label==1,t==1)[t==1])

    if is_fp_tp:
        return np.concatenate(data), np.concatenate(file_number), np.concatenate(fps), np.concatenate(tps)
        
    return np.concatenate(data), np.concatenate(file_number)

In [ ]:
files = range(5,15)#[5,6,7,9,13,14]
data,file_num, ano_fps, ano_tps = get_anomalous_classified_window(best_model_vae,threshold_vae, files=files,is_fp_tp=True)

In [ ]:


# method 2 using feature wise threshold

# testing option 1
test_data = []
labels_1 = []
file_number = []
# m = model_ext


files = os.listdir(data_pth+test_1)

print(files)
for file in files:
    tmp = pd.read_csv(data_pth+test_1+'/'+file, sep=';')
    t = tmp.anomaly.values
    test_df_1_norm, _  = min_max_normalize(tmp.drop(columns=['datetime','anomaly','changepoint']).values,new_m_m_parms)
    test_x = create_time_window(test_df_1_norm)
    # pred_x = model_ext.predict(test_x)
    labels_1.append(t[-len(test_x):])
    i = int(file.split('.')[0])
    file_number.append([i]*len(test_x))
    test_data.append(test_x)
    # preds.append(anomaly_score_no_mean(np.squeeze(pred_x),np.squeeze(test_x)).mean(axis=1)>threshold)
    # preds.append(np.any(anomaly_score_no_mean(np.squeeze(pred_x),np.squeeze(test_x)).mean(axis=1)>threshold_feature, axis=1))
    # break
    
# preds = np.concatenate(preds, axis=0)
test_data = np.concatenate(test_data)
labels_1 = np.concatenate(labels_1, axis=0)
file_number = np.concatenate(file_number, axis=0)


dict_to_save = {'ano_data':data, 'cf_data':cf_anom, 'file_num':file_number }
dict_to_save['test_data']=test_data
dict_to_save['test_labels']=labels_1
dict_to_save['test_file_num']=file_number

dict_to_save['train_data']=train_ext_x

import pickle as pk
pk.dump(dict_to_save, open('skab_vae_grad.pk','wb'))

In [ ]:
labels_1

In [ ]:
import os
import pickle
import umap
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
# Check current working directory
print("Current working directory:", os.getcwd())

# Path to the pickle file
file_path_skab = 'skab_vae_grad.pk' 

# Attempt to open the file with error handling
try:
    with open(file_path_skab, 'rb') as file:
        var_skab = pickle.load(file)
    print("File loaded successfully")
except FileNotFoundError:
    print(f"File not found: {file_path_skab}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
anomaly_data_skab = var_skab['ano_data']
ano_file_skab = var_skab['file_num']
cf_data_skab = var_skab['cf_data']

train_data_skab = var_skab['train_data']
test_data_skab = var_skab['test_data']
combined_file_num_skab = var_skab['test_file_num']

index_counts_skab = Counter(combined_file_num_skab)

In [ ]:
exclude_indices = index_counts_skab[1] + index_counts_skab[2] + index_counts_skab[3] + index_counts_skab[4]

In [ ]:
exclude_ind_test_data = test_data_skab[exclude_indices:,:,:]

In [ ]:
np.count_nonzero(var_skab['test_labels'][exclude_indices:])

In [ ]:
len(list(var_skab['test_labels'][exclude_indices:]))

In [ ]:
labels_train_combined_skab = np.array([0]*train_data_skab.shape[0]+ list(var_skab['test_labels'][exclude_indices:]))

In [ ]:
def plot_umap_projections(train_data, combined_test_data, anomaly_data, cf_data, labels_train_combined, plot_name):
    
    X_train_flat = train_data.reshape(train_data.shape[0], -1) if train_data.ndim > 2 else train_data
    X_combined_flat = combined_test_data.reshape(combined_test_data.shape[0], -1) if combined_test_data.ndim > 2 else combined_test_data
    X_anom_flat = anomaly_data.reshape(anomaly_data.shape[0], -1) if anomaly_data.ndim > 2 else anomaly_data
    counterfactuals_flat = cf_data.reshape(cf_data.shape[0], -1) if cf_data.ndim > 2 else cf_data
    
    # Combine train and test data
    X_train_combined = np.concatenate([X_train_flat, X_combined_flat], axis=0)
    
    # Initialize UMAP reducer
    umap_reducer = umap.UMAP(n_neighbors=30, min_dist=0.5, n_components=2, metric='euclidean')
    
    # Transform data
    X_reduced = umap_reducer.fit_transform(X_train_combined)
    anomaly_reduced = umap_reducer.transform(X_anom_flat)
    counterfactuals_reduced = umap_reducer.transform(counterfactuals_flat)
    
    # Plotting
    plt.figure(figsize=(10, 8))
    plt.scatter(X_reduced[labels_train_combined == 0, 0], X_reduced[labels_train_combined == 0, 1], label='Normal', alpha=0.5, color='limegreen')
    plt.scatter(X_reduced[labels_train_combined == 1, 0], X_reduced[labels_train_combined == 1, 1], label='Anomaly', alpha=0.5, color='crimson')
    # plt.scatter(anomaly_reduced[:, 0], anomaly_reduced[:, 1], label='Anomaly', alpha=0.5, color='crimson')
    plt.scatter(counterfactuals_reduced[:, 0], counterfactuals_reduced[:, 1], label='Counterfactual', alpha=0.3, color='gold', edgecolor='k', marker='o', s=40)
    plt.xlabel('UMAP Dimension 1', fontsize=20)
    plt.ylabel('UMAP Dimension 2', fontsize=20)
    plt.legend(loc='lower left', prop={'size': 15})
    plt.savefig(f'umap_{plot_name}_grad_vae.png', dpi=1000)
    plt.show()

In [ ]:
plot_umap_projections(train_data=train_data_skab, 
                      combined_test_data=exclude_ind_test_data, 
                      anomaly_data=anomaly_data_skab, 
                      cf_data=cf_data_skab, 
                      labels_train_combined=labels_train_combined_skab, 
                      plot_name='skab')

In [ ]:
sensor_names = ['Sensor_' + str(i) for i in range(test_loss_mae_mse_ae.shape[1])]
plt.figure(figsize=(6, 6))
heatmap = sns.heatmap(np.swapaxes(test_loss_mae_mse_ae,1,2)[400], xticklabels=sensor_names, cmap="viridis")
heatmap.set_title('Reconstruction Losses(Window Based) Heatmap')
heatmap.set_xlabel('Number of Features/Sensors')
heatmap.set_ylabel('Window length')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('recon_errors.png', bbox_inches='tight', dpi=1000)
plt.show()

In [ ]:
def reconstruct_from_tensor(tensor):
    # Assuming tensor shape is (num_sequences, num_features, sequence_length)
    # And the sequences are overlapping with step size of 1
    
    # Initialize an empty list to hold the reconstructed data
    reconstructed_data = []
    
    # Convert tensor to numpy for easier manipulation if it's a torch tensor
    if isinstance(tensor, torch.Tensor):
        tensor = tensor.numpy()
    
    num_sequences, num_features, sequence_length = tensor.shape
    
    # Iterate through the sequences
    for i in range(num_sequences):
        # For all but the last sequence, take the first element
        if i < num_sequences - 1:
            reconstructed_data.append(tensor[i, :, 0])
        else:
            # For the last sequence, take all elements to ensure we don't miss the tail of the dataset
            reconstructed_data.extend(tensor[i, :, :].transpose())

    # Convert the list to a numpy array
    reconstructed_array = np.array(reconstructed_data)
    
    # Reshape the array back to 2D (num_samples, num_features)
    # This step may not be necessary depending on how reconstructed_data is structured
    
    # Convert to DataFrame
    df_reconstructed = pd.DataFrame(reconstructed_array, columns=sensor_names, dtype=np.float64)
    
    return df_reconstructed

# reconstruct_from_tensor(cf_anom)

In [ ]:
cf_dataframe = reconstruct_from_tensor(cf_anom)

In [ ]:
train_data_normalized = reconstruct_from_tensor(train_ext_x.swapaxes(1,2))

In [ ]:
train_ext_x.swapaxes(1,2).shape

In [ ]:
feature_names = df.drop(columns=['datetime']).columns

In [ ]:
feature_names

In [ ]:
[2 3 5]

In [ ]:
test_df = pd.DataFrame(test_df_1_norm)

In [ ]:
cf_dataframe.columns = feature_names
test_df.columns = feature_names
train_data_normalized.columns = feature_names

In [ ]:
from data_drift_detector import DataDriftDetector
detector = DataDriftDetector(df_prior = train_data_normalized, df_post = test_df)
detector.calculate_drift()
detector.plot_numeric_to_numeric(plot_numeric_columns=['Accelerometer1RMS', 'Accelerometer2RMS'])
plt.savefig('driftdetector_grad_vae_skab.png', dpi=1000)

In [ ]:
detector = DataDriftDetector(df_prior = train_data_normalized, df_post = cf_dataframe)
detector.calculate_drift()
detector.plot_numeric_to_numeric(plot_numeric_columns=['Accelerometer1RMS', 'Accelerometer2RMS'])
plt.savefig('driftdetector_grad_vae_skab_cf.png', dpi=1000)